In [1]:
import gazpacho

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_world_records_in_swimming"
html = gazpacho.get(URL)

In [3]:
print(len(html))

572584


In [4]:
html[:500]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 skin-theme-clientpref-day vector-sticky-header-enabled vector-to'

In [6]:
html[-500:]

'anization","name":"Contributors to Wikimedia projects"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\\/\\/www.wikimedia.org\\/static\\/images\\/wmf-hor-googpub.png"}},"datePublished":"2007-03-15T21:20:10Z","dateModified":"2026-01-17T08:19:18Z","image":"https:\\/\\/upload.wikimedia.org\\/wikipedia\\/commons\\/a\\/ae\\/Caeleb_Dressel_before_winning_100_fly_%2842769914221%29.jpg","headline":"Wikimedia list article"}</script>\n</body>\n</html>'

In [13]:
table_start = html.find("<table")
html[table_start:table_start+500]

'<table class="wikitable sortable" style="font-size: 95%;">\n<caption>\n</caption>\n<tbody><tr>\n<th>Event\n</th>\n<th style="width:4em" class="unsortable">Time\n</th>\n<th class="unsortable">\n</th>\n<th>Name</th>\n<th>Nationality</th>\n<th>Date</th>\n<th>Meet</th>\n<th>Location\n</th>\n<th class="unsortable">Ref\n</th></tr>\n\n<tr>\n<td><span data-sort-value="01&#160;!"><a href="/wiki/World_record_progression_50_metres_freestyle" title="World record progression 50 metres freestyle">50m freestyle</a></span>\n</td>\n<'

In [14]:
soup = gazpacho.Soup(html)

In [15]:
type(soup)

gazpacho.soup.Soup

In [25]:
tables = soup.find("table")

In [29]:
for n, table in enumerate(tables):
    rows = table.find("tr", mode="all")
    cols = rows[-1].find("td", mode="all")
    print(f"{n}->{len(rows)} rows,{len(cols)} columns")

0->21 rows,9 columns
1->21 rows,9 columns
2->3 rows,9 columns
3->25 rows,9 columns
4->27 rows,9 columns
5->4 rows,9 columns
6->25 rows,11 columns
7->1 rows,2 columns
8->7 rows,1 columns
9->5 rows,1 columns
10->8 rows,1 columns
